# Interface

In [ ]:
!pip install "jinja2<3.1"
!pip install --upgrade Flask
!pip install flask_cors


  Using cached Jinja2-3.0.3-py3-none-any.whl.metadata (3.5 kB)
Using cached Jinja2-3.0.3-py3-none-any.whl (133 kB)
  Attempting uninstall: jinja2
    Found existing installation: Jinja2 3.1.6
    Uninstalling Jinja2-3.1.6:
      Successfully uninstalled Jinja2-3.1.6


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
anaconda-project 0.10.1 requires ruamel-yaml, which is not installed.
spyder 5.1.5 requires pyqtwebengine<5.13, which is not installed.
cookiecutter 1.7.2 requires Jinja2<3.0.0, but you have jinja2 3.0.3 which is incompatible.
cookiecutter 1.7.2 requires MarkupSafe<2.0.0, but you have markupsafe 2.1.3 which is incompatible.
flask 3.1.2 requires jinja2>=3.1.2, but you have jinja2 3.0.3 which is incompatible.
nbclassic 0.2.6 requires jupyter-server~=1.1, but you have jupyter-server 2.8.0 which is incompatible.
nbclassic 0.2.6 requires notebook<7, but you have notebook 7.0.6 which is incompatible.
pandasai 2.3.2 requires jinja2<4.0.0,>=3.1.3, but you have jinja2 3.0.3 which is incompatible.
spyder 5.1.5 requires pyqt5<5.13, but you have pyqt5 5.15.10 which is incompatible.

[notice] A new release of pip is available:

## Servidor

In [ ]:
from flask import Flask, request, jsonify
from flask_cors import CORS
import os, pickle, re
import numpy as np
import pandas as pd
import networkx.drawing.nx_pydot as nx_pydot
import threading

def sanitize(obj):
    if isinstance(obj, pd.Series):
        return obj.tolist()
    elif isinstance(obj, pd.DataFrame):
        return obj.to_dict(orient='records')
    elif isinstance(obj, np.generic):
        return obj.item()
    elif isinstance(obj, dict):
        return {k: sanitize(v) for k, v in obj.items()}
    elif isinstance(obj, list):
        return [sanitize(v) for v in obj]
    else:
        return obj

app = Flask(__name__)
CORS(app)

def _strip_quotes(s):
    if not isinstance(s, str):
        return s
    s = s.strip()
    if len(s) >= 2 and ((s[0] == s[-1] == '"') or (s[0] == s[-1] == "'")):
        return s[1:-1]
    return s

@app.route('/upload', methods=['POST'])
def upload_dot():
    if 'dot' not in request.files:
        return jsonify({'error': 'Faltou o ficheiro .dot (campo "dot").'}), 400

    file = request.files['dot']
    filename = file.filename or 'graph.dot'

    os.makedirs('uploads', exist_ok=True)
    dot_path = os.path.join('uploads', filename)
    file.save(dot_path)

    with open(dot_path, encoding='iso-8859-1') as f:
        dot_code = f.read()

    G = nx_pydot.read_dot(dot_path)

    pkl_filename = G.graph.get('variables_file')
    pkl_filename = _strip_quotes(pkl_filename) if pkl_filename else None

    if not pkl_filename:
        m = re.search(r'//\s*variables_file\s*=\s*([^\s]+)', dot_code)
        if m:
            pkl_filename = _strip_quotes(m.group(1))

    if not pkl_filename:
        return jsonify({'error': 'Não foi possível encontrar "variables_file" no DOT.'}), 400

    if os.path.isabs(pkl_filename):
        pkl_path = pkl_filename
    else:
        pkl_path = os.path.join(os.path.dirname(dot_path), pkl_filename)
        if not os.path.exists(pkl_path) and os.path.exists(pkl_filename):
            pkl_path = pkl_filename

    if not os.path.exists(pkl_path):
        return jsonify({'error': f'Ficheiro PKL não encontrado: {pkl_filename} (resolvido para {pkl_path})'}), 400

    with open(pkl_path, 'rb') as f:
        vars_loaded = pickle.load(f)

    if isinstance(vars_loaded.get('df_final'), (pd.Series, pd.DataFrame)):
        vars_loaded['df_final'] = vars_loaded['df_final'].tolist()

    vars_loaded = sanitize(vars_loaded)

    n_nodes = len(G.nodes())
    n_edges = len(G.edges())

    utterances_in_dataset = 0
    speaker_cluster_counts = vars_loaded.get('speaker_cluster_counts', {})
    if isinstance(speaker_cluster_counts, dict):
        for s, counts in speaker_cluster_counts.items():
            try:
                utterances_in_dataset += int(sum(counts))
            except Exception:
                pass

    df_final_raw = vars_loaded.get('df_final', [])
    try:
        s_df = pd.Series(df_final_raw)
        dialogues_counts = s_df.value_counts()
        n_dialogues_in_dataset = int(dialogues_counts.shape[0])
        utterances_per_dialogue_mean = float(dialogues_counts.mean()) if len(dialogues_counts) else 0.0
        utterances_per_dialogue_std = float(dialogues_counts.std(ddof=0)) if len(dialogues_counts) > 1 else 0.0
    except Exception:
        n_dialogues_in_dataset = len(set(df_final_raw)) if df_final_raw else 0
        utterances_per_dialogue_mean = 0.0
        utterances_per_dialogue_std = 0.0

    fsc = vars_loaded.get('fsc_speaker', {})
    try:
        fsc_vals = [float(v) for v in (fsc.values() if isinstance(fsc, dict) else fsc)]
        fsc_mean = float(sum(fsc_vals) / len(fsc_vals)) if fsc_vals else 0.0
    except Exception:
        fsc_mean = 0.0

    avg_EOD = float(vars_loaded.get('avg_EOD', 0.0))
    avg_SOD = float(vars_loaded.get('avg_SOD', 0.0))
    density = float(vars_loaded.get('density', 0.0))
    sent_variation = avg_EOD - avg_SOD

    return jsonify({
        'dot_code': dot_code,
        'speakers': vars_loaded.get('speakers', []),
        'speaker_cluster_counts': vars_loaded.get('speaker_cluster_counts', {}),
        'fsc_speaker': vars_loaded.get('fsc_speaker', {}),
        'avg_EOD': avg_EOD,
        'avg_SOD': avg_SOD,
        'df_final': vars_loaded.get('df_final', []),
        'density': density,
        'clusters_speaker': vars_loaded.get('clusters_speaker', {}),
        'diretoria': vars_loaded.get('diretoria', ""),
        'cluster_id_speaker': vars_loaded.get('cluster_id_speaker', {}),
        'n_nodes': n_nodes,
        'n_edges': n_edges,
        'utterances_in_dataset': utterances_in_dataset,
        'n_dialogues_in_dataset': n_dialogues_in_dataset,
        'utterances_per_dialogue_mean': utterances_per_dialogue_mean,
        'utterances_per_dialogue_std': utterances_per_dialogue_std,
        'fsc_mean': fsc_mean,
        'sent_variation': sent_variation
    })

def run_flask():
    app.run(debug=False, use_reloader=False)

thread = threading.Thread(target=run_flask)
thread.start()


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit


In [1]:
import sys
sys.exit(0)

SystemExit: 0

c:\Users\tixap\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3452: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


## Frontend

In [5]:
import pickle
import os
import networkx.drawing.nx_pydot as nx_pydot

def extract_variables_file_from_dot(dot_filepath: str) -> str:
    with open(dot_filepath, 'r', encoding='utf-8') as f:
        for line in f:
            if line.strip().startswith("// variables_file="):
                return line.strip().split("=", 1)[1]
    raise ValueError("variável 'variables_file' não encontrada no .dot")

def load_data_from_dot(dot_filepath: str):
    global speakers, speaker_cluster_counts, fsc_speaker
    global avg_EOD, avg_SOD, df_final, density
    global clusters_speaker, diretoria, cluster_id_speaker
    global dot_code, result

    result = nx_pydot.read_dot(dot_filepath)
    pkl_filename = extract_variables_file_from_dot(dot_filepath)
    
    if pkl_filename is None:
        raise ValueError("O ficheiro .dot não contém a referência ao ficheiro .pkl.")

    dot_dir = os.path.dirname(dot_filepath)
    pkl_filepath = os.path.join(dot_dir, pkl_filename)

    with open(pkl_filepath, "rb") as f:
        vars_loaded = pickle.load(f)

    speakers = vars_loaded["speakers"]
    speaker_cluster_counts = vars_loaded["speaker_cluster_counts"]
    fsc_speaker = vars_loaded["fsc_speaker"]
    avg_EOD = vars_loaded["avg_EOD"]
    avg_SOD = vars_loaded["avg_SOD"]
    df_final = vars_loaded["df_final"]
    density = vars_loaded["density"]
    clusters_speaker = vars_loaded["clusters_speaker"]
    diretoria = vars_loaded["diretoria"]
    cluster_id_speaker = vars_loaded["cluster_id_speaker"]

    with open(dot_filepath, encoding='iso-8859-1') as f:
        dot_code = f.read()

    return {
        "speakers": speakers,
        "speaker_cluster_counts": speaker_cluster_counts,
        "fsc_speaker": fsc_speaker,
        "avg_EOD": avg_EOD,
        "avg_SOD": avg_SOD,
        "df_final": df_final,
        "density": density,
        "clusters_speaker": clusters_speaker,
        "diretoria": diretoria,
        "cluster_id_speaker": cluster_id_speaker,
        "dot_code": dot_code,
        "graph_result": result,
    }


In [6]:
import os
import webbrowser
import json 
import json
import numpy as np
import networkx as nx


def atribuir_cor(speaker):
    cores_fixas = {
        "Rachel": "#FF6B6B",  
        "Joey": "#FF9966",     
        "Monica": "#66CDAA",    
        "Ross": "#6A5ACD",      
        "Chandler": "#B266FF", 
        "Phoebe": "#FF69B4",    
        "Others": "#40E0D0",     
        "User": "#33CCFF",
        "System": "#3300FF",
        "SOD": "#FFD700",     
        "EOD": "#FFD700"      
    }

    if speaker in cores_fixas:
        return cores_fixas[speaker]

    if speaker in cores_personalizadas:
        return cores_personalizadas[speaker]

    
    cores_usadas = set(cores_fixas.values()).union(cores_personalizadas.values())
    while True:
        cor_aleatoria = "#" + "".join(random.choices("0123456789ABCDEF", k=6))
        if cor_aleatoria not in cores_usadas:
            cores_personalizadas[speaker] = cor_aleatoria
            return cor_aleatoria

speakers = []
speaker_cluster_counts = []
fsc_speaker = None
avg_EOD = None
avg_SOD = None
df_final = []
density = []
clusters_speaker = []
diretoria = ""
cluster_id_speaker = []
dot_code = ""
result = None

file_name_inicial = "DAs_train_emo_none_NONE_1_graph.dot"
data = load_data_from_dot(file_name_inicial)

n_nos = len(result.nodes)
n_edges = len(result.edges)


utterances_in_dataset=0
for speaker in speakers:        
    utterances_in_dataset += speaker_cluster_counts[speaker].sum() 

n_dialogues_in_dataset = df_final.nunique()
utterances_per_dialogue_list = df_final.value_counts()
utterances_per_dialogue_mean = utterances_per_dialogue_list.mean()
utterances_per_dialogue_std = utterances_per_dialogue_list.std()
fsc_aux=0
for speaker in speakers: 
    fsc_aux += fsc_speaker[speaker]
fsc_mean = fsc_aux / len(speakers)

sent_variation = avg_EOD - avg_SOD

thresholds = np.arange(0.0, 1.01, 0.1)
threshold_values = [round(t, 2) for t in thresholds]
threshold_data = {
    f"{t:.2f}": {
        "shapes": "",
        "annotations": "" 
    }
    for i, t in enumerate(thresholds)
}

threshold_json = json.dumps(threshold_data)


dot_code_js = json.dumps(dot_code)


checkboxes_html = """
    <div style="display: grid; grid-template-columns: repeat(2, 1fr); gap: 5px;">
        <div>
            <input type="radio" id="todos" name="speaker" value="__all__" checked>
            <label for="todos" style="color: white;">Todos</label>
        </div>
"""

for speaker in speakers:
    speaker_id = speaker.replace(" ", "_").lower()
    cor = atribuir_cor(speaker)
    checkboxes_html += f"""
        <div>
            <input type="radio" id="{speaker_id}" name="speaker" value="{speaker}" >
            <label for="{speaker_id}" style="color: {cor};">{speaker}</label>
        </div>
    """

checkboxes_html += "</div>"


metricas_html = f"""
<div style="padding-left: 10px; font-size: 14px; line-height: 1.6;">
    <h3 style="margin-bottom: 5px;">Dialogue Metrics</h3>
    Utterances in Dataset: {utterances_in_dataset}<br>
    Dialogues in Dataset: {n_dialogues_in_dataset}<br>
    Utterances per Dialogue: {utterances_per_dialogue_mean:.2f} ± {utterances_per_dialogue_std:.2f}<br><br>

    <h3 style="margin-bottom: 5px;">Clustering Metrics</h3>
    Number of States: {n_nos}<br>
    Sentiment Cohesion: {fsc_mean:.2f}<br><br>

    <h3 style="margin-bottom: 5px;">Flow Metrics</h3>
    Number of Transitions: {n_edges}<br>
    Initial Sentiment: {avg_SOD:.2f}<br>
    Final Sentiment: {avg_EOD:.2f}<br>
    Sentiment Variation: {sent_variation:.2f}<br>
    Flow Density: {density:.2f}
</div>
"""


html_content = f"""
<!DOCTYPE html>
<html lang="pt-BR">
<head>
    <meta charset="UTF-8" />
    <link href="https://fonts.googleapis.com/css2?family=Lato:wght@300;400;700&display=swap" rel="stylesheet">
    <script src="https://unpkg.com/viz.js@2.1.2/viz.js"></script>
    <script src="https://unpkg.com/viz.js@2.1.2/full.render.js"></script>
    <script src="https://cdn.plot.ly/plotly-latest.min.js"></script>
    <script src="https://cdnjs.cloudflare.com/ajax/libs/html2canvas/1.4.1/html2canvas.min.js"></script>
    <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/font-awesome/6.5.0/css/all.min.css">
    <style>
        body {{
            margin: 0;
            font-family: 'Lato', sans-serif;
        }}
        .custom-tooltip {{
            position: absolute;
            background-color: #8B0000;
            color: white;
            padding: 10px;
            border-radius: 6px;
            font-size: 14px;
            max-width: 400px;
            white-space: pre-wrap;
            pointer-events: none;
            z-index: 10000;
            display: none;
            box-shadow: 2px 2px 6px rgba(0,0,0,0.6);
        }}
        #sidebar {{
            height: 100%;
            width: 0;
            position: fixed;
            z-index: 999;
            top: 0;
            left: 0;
            background-color: #002366;
            overflow-x: hidden;
            transition: 0.3s;
            padding-top: 60px;
            color: white;
            box-shadow: 2px 2px 6px rgba(0,0,0,0.6);
        }}
        #selection-rectangle {{
            position: absolute;
            border: 2px dashed #007bff;
            background-color: rgba(0, 123, 255, 0.2);
            pointer-events: none;
            display: none;
            z-index: 9999;
            }}
        #openSidebar {{
            position: fixed;
            top: 20px;
            left: 0;
            z-index: 1000;
            background-color: #002366;
            color: white;
            padding: 10px 15px;
            cursor: pointer;
            border-top-right-radius: 8px;
            border-bottom-right-radius: 8px;
        }}
        #sidebarContent {{
            padding: 15px;
            overflow-y: auto;
            max-height: 90%;
        }}
        .closebtn {{
            position: absolute;
            top: 10px;
            right: 20px;
            font-size: 30px;
            cursor: pointer;
        }}
        #grafico {{
            margin-left: 0;
            transition: margin-left 0.3s;
            padding: 10px;
            overflow: hidden;
            
        }}
        h1#main-title {{
            position : sticky;
            width: 100%;
            margin-left: -15px;
            background: white;
            z-index: 1;
            color: #002366; 
            text-align: center;
            font-weight: 700;
            margin-top: -10px;
            margin-bottom: 20px;
            padding-top: 30px;   
            padding-bottom: 15px;
            padding-right: 35px;
            font-size: 3rem;
            font-family: 'Lato', sans-serif;
        }}
        h2, h3 {{
            font-weight: bold;
            color: #fff;
        }}
        input[type=range] {{
            width: 90%;
        }}
        .box {{
            padding: 15px;
            margin-bottom: 20px;
            border-radius: 10px;
            box-shadow: 3px 3px 8px rgba(0,0,0,0.8);
            font-size: 14px;
            line-height: 1.6;
            
        }}
        svg {{
            width: 100%;
            height: 80vh;
        }}
        .selected {{
            stroke: #87CEEB !important;
            stroke-width: 3px !important;
        }}
        .hidden {{
            display: none !important;
        }}
        .faded {{
            opacity: 0.2 !important;
        }}
        /* Slider customizado */
        input[type=range] {{
            -webkit-appearance: none;
            width: 90%;
            height: 9px;
            background: #ddd;
            border-radius: 10px;
            outline: none;
            cursor: pointer;
        }}
        input[type=range]::-webkit-slider-thumb {{
            -webkit-appearance: none;
            appearance: none;
            width: 20px;
            height: 20px;
            background: #87CEEB;
            cursor: pointer;
            border-radius: 10px;
            border: 2px solid #999;
            margin-top: -3px;
            transition: background 0.3s ease;
            border-color: #0055aa;
        }}
        input[type=range]:hover::-webkit-slider-thumb {{
            background: ##87CEEB;
            border-color: #0055aa;
        }}
        input[type=range]::-moz-range-thumb {{
            width: 20px;
            height: 20px;
            background: #87CEEB;
            border-radius: 10px;
            border: 2px solid #999;
            cursor: pointer;
        }}
        /* Radio buttons */
        input[type="radio"] {{
            accent-color: #87CEEB;
            width: 18px;
            height: 18px;
            cursor: pointer;
        }}
        g.node title {{
            pointer-events: none;
        }}
        #graph-controls button {{
            background-color: #002366;
            border: none;
            color: white;
            padding: 0px;
            margin-right: 5px;
            cursor: pointer;
            border-radius: 5px;
            transition: background-color 0.3s ease;
        }}
        #graph-controls button:hover {{
            background-color: #0055aa ;
        }}
        #graph-wrapper {{
            position: relative;
        }}
        box1 {{
            padding: 15px;
            margin-bottom: 20px;
            border-radius: 10px;
            box-shadow: 3px 3px 8px rgba(0,0,0,0.8);
            font-size: 14px;
            line-height: 1.6;
            display: flex;
            justify-content: center;
            gap: 5px;
            margin-bottom: 40%;
        }}
        .box1 button {{
            width: 32.5px;
            height: 32.5px;
            background-color: #87CEEB
            color: white; 
            font-size: 20px;
            border: none;
            border-radius: 8px;
            box-shadow: 0 4px 8px rgba(0, 0, 0, 0.2);
            cursor: pointer;
            transition: transform 0.1s, box-shadow 0.1s;
        }}
        .box1 button:hover {{
            transform: translateY(-2px);
            box-shadow: 0 6px 12px rgba(0, 0, 0, 0.3);
        }}
        .box1 button:active {{
            transform: scale(0.95);
            box-shadow: 0 2px 4px rgba(0, 0, 0, 0.2);
        }}
        #searchInput:focus {{
            border-color: #87CEEB;
            outline: none;
        }}
        .collapsible-header {{
            cursor: pointer;
            display: inline-flex;
            justify-content: space-between;
            align-items: center;
            border-radius: 8px;
            user-select: none;
            margin-bottom: 10px;
            gap: 100px;
            margin-top: 1px;
            margin-bottom: 1px;
        }}
        .collapsible-header1 {{
            cursor: pointer;
            display: inline-flex;
            justify-content: space-between;
            align-items: center;
            border-radius: 8px;
            user-select: none;
            margin-bottom: 10px;
            gap: 100px;
            margin-top: 1px;
            margin-bottom: 1px;
        }}
        .collapsible-content {{
            display: block;
            overflow: hidden;
            transition: max-height 0.3s ease;
        }}

        .collapsible.collapsed .collapsible-content {{
            display: none;
        }}

        .collapsible .arrow {{
            width: 12px;
            height: 12px;
            display: inline-block;
            border-left: 2px solid white;
            border-bottom: 2px solid white;
            transform: rotate(45deg);
            transition: transform 0.3s ease;
            margin-left: 8px;
        }}
        .collapsible.collapsed .arrow {{
            transform: rotate(-45deg); 
        }}
        .collapsible-header {{
            display: inline-flex;
            align-items: center;
            gap: 100px; 
        }}
        .collapsible-header1 {{
            display: inline-flex;
            align-items: center;
            gap: 117px; 
        }}
        .collapsible-header .arrow {{
            margin-left: 0; 
            font-size: 0.8em; 
        }}
</style>
</head>

<body>
<div id="openSidebar" onclick="openSidebar()">☰</div>

<div id="sidebar">
    <span class="closebtn" onclick="closeSidebar()">×</span>
    
    <div id="sidebarContent">
        <div class="box">
            <h2 style="margin-top:-1px">Upload File</h2>
            <input type="file" id="dotFileInput" accept=".dot" style="display: none;">
            <label 
                for="dotFileInput" 
                id="fileLabel"
                style="
                    display: flex;
                    flex-direction: row;
                    align-items: center;
                    justify-content: center;
                    gap: 8px;
                    padding: 10px 16px;
                    background-color: rgb(0 35 102);
                    border-radius: 6px;
                    cursor: pointer;
                    color: white;
                    font-size: 14px;
                    font-weight: bold;
                    transition: background-color 0.3s ease;
                    height: 40px;
                    width: 100%;
                    box-sizing: border-box;
                    box-shadow: 3px 3px 8px rgba(0,0,0,0.8);
                    overflow: hidden;
                "
                onmouseover="this.style.backgroundColor='#0055aa'"
                onmouseout="this.style.backgroundColor='#002366'"
            >
            <svg xmlns="http://www.w3.org/2000/svg" fill="white" viewBox="0 0 24 24" width="16" height="16" style="flex-shrink: 0; width: 15%;" >
                <path d="M5 20h14v-2H5v2zm7-14l5 5h-3v4h-4v-4H7l5-5z"/>
            </svg>
            <span id="fileLabelText" 
                style="
                    display: inline-block;
                    max-width: calc(100% - 30px); 
                    white-space: nowrap;
                    overflow: hidden;
                    text-overflow: ellipsis;
                "
            >Choose file</span>
            </label>

        </div>
        
        <div class="box">
            <div id="graph-controls" class="box1"> 
                <button id="btn-reset" title="Resetar gráfico"><i class="fas fa-home" style="color:white;"></i></button>
                <button id="btn-zoom-in" title="Zoom In"><i class="fas fa-search-plus" style="color:white;"></i></button>
                <button id="btn-zoom-out" title="Zoom Out"><i class="fas fa-search-minus" style="color:white;"></i></button>
                <button id="btn-screenshot" title="Salvar Print"><i class="fas fa-camera" style="color:white;"></i></button>
                <button id="btn-pan" title="Mover grafo (modo mão)"><i class="fas fa-hand-paper" style="color:white;"></i></button>
                <button id="btn-zoom-box" title="Zoom em área selecionada">
                    <i class="fas fa-object-group" style="color:white;"></i>
                </button>
            </div>
        </div>
        <div class="box">
        <div class="collapsible collapsed">
            <h2 class="collapsible-header">Parâmetros <span class="arrow"></span></h2>
            <div class="collapsible-content">
                <h3>Search Nodes</h3>
                <input type="text" id="searchInput" placeholder="Enter a word..." style="width: 90%; padding: 5px; border: 2px solid #87CEEB; border-radius: 10px;">

                <h3>Speakers</h3>
                {checkboxes_html}

                <h3>Threshold</h3>
                <input type="range" id="thresholdSlider" min="0" max="100" value="0" step="1" oninput="updateThreshold(this.value)">
                <div style="color: white; margin-top: 5px;">Valor: <span id="thresholdLabel">0.00</span></div>
            </div>
        </div>
        </div>
        <div class="box">
            <div class="collapsible collapsed">
            <h2 class="collapsible-header1">Métricas <span class="arrow"></span></h2>
            <div class="collapsible-content" id="metrics-content">
                {metricas_html}
    </div>
</div>
        </div>
    </div>
</div>

<div id="grafico">
    <h1 id="main-title">FlowDisco</h1>
    <p id="file-subtitle" style="text-align: center; color: #002366; font-family: 'Lato', sans-serif; margin-top: -23px; margin-bottom: 10px; margin-left: -10px;"></p>
    <div id="graph-wrapper" style="overflow: auto; width: 100%;">
        <div id="graph-container"></div>
    </div>
</div>


<div id="tooltip" class="custom-tooltip"></div>

<script>
    
    function makeNodesDraggable(svgElement) {{
        let selectedNode = null;
        let offsetX, offsetY;

        function getNodeCenter(node) {{
            const transform = node.getAttribute("transform");
            const match = /translate\(([^,]+),([^)]+)\)/.exec(transform);
            if (match) {{
                return {{ x: parseFloat(match[1]), y: parseFloat(match[2]) }};
            }}
            return {{ x: 0, y: 0 }};
        }}

        function updateConnectedEdges(node) {{
            const nodeName = node.querySelector("title")?.textContent.trim();
            if (!nodeName) return;

            svgElement.querySelectorAll("g.edge").forEach(edge => {{
                const edgeTitle = edge.querySelector("title")?.textContent.trim();
                if (!edgeTitle) return;

                if (edgeTitle.includes(nodeName)) {{
                    const parts = edgeTitle.split("->").map(s => s.trim());
                    if (parts.length >= 2) {{
                        const sourceNode = Array.from(svgElement.querySelectorAll("g.node"))
                            .find(n => n.querySelector("title")?.textContent.trim() === parts[0]);
                        const targetNode = Array.from(svgElement.querySelectorAll("g.node"))
                            .find(n => n.querySelector("title")?.textContent.trim() === parts[1]);

                        if (sourceNode && targetNode) {{
                            const sourcePos = getNodeCenter(sourceNode);
                            const targetPos = getNodeCenter(targetNode);
                            const path = edge.querySelector("path");
                            if (path) {{
                                path.setAttribute("d", `M${{sourcePos.x}} ${{sourcePos.y}} L${{targetPos.x}} ${{targetPos.y}}`);
                            }}
                        }}
                    }}
                }}
            }});
        }}

        svgElement.querySelectorAll("g.node").forEach(node => {{
            node.addEventListener("mousedown", e => {{
                selectedNode = node;
                const pos = getNodeCenter(node);
                offsetX = e.clientX - pos.x;
                offsetY = e.clientY - pos.y;
                e.preventDefault();
            }});
        }});

        svgElement.addEventListener("mousemove", e => {{
            if (!selectedNode) return;
            const x = e.clientX - offsetX;
            const y = e.clientY - offsetY;
            selectedNode.setAttribute("transform", `translate(${{x}},${{y}})`);
            updateConnectedEdges(selectedNode);
            e.preventDefault();
        }});

        svgElement.addEventListener("mouseup", e => {{
            selectedNode = null;
            e.preventDefault();
        }});

        svgElement.addEventListener("mouseleave", e => {{
            selectedNode = null;
        }});
    }}







    function enviarFicheiro() {{
        const input = document.getElementById('dotFileInput');
        const file = input.files[0];
        document.getElementById("file-subtitle").textContent = "Dialogue Flow Discovery for: " + file.name;
        
        
        if (!file) return alert("Selecione um ficheiro .dot");

        const formData = new FormData();
        formData.append("dot", file);

        fetch("http://localhost:5000/upload", {{
            method: "POST",
            body: formData
        }})
        .then(res => res.json())
        .then(data => {{
            renderNewDot(data.dot_code);
            
            const metrics = document.getElementById("metrics-content");
            const num = (x, d=2) => isFinite(Number(x)) ? Number(x).toFixed(d) : "0.00";
            const int = (x) => Number.isFinite(Number(x)) ? Number(x) : 0;

            metrics.innerHTML = `
            <h3 style="margin-bottom:5px;">Dialogue Metrics</h3>
            Utterances in Dataset: ${{int(data.utterances_in_dataset) || (
                Array.isArray(data.speakers) && data.speaker_cluster_counts
                ? data.speakers.reduce((acc, s) => {{
                    const arr = data.speaker_cluster_counts[s] || [];
                    return acc + arr.reduce((a, b) => a + Number(b || 0), 0);
                    }}, 0)
                : 0
            )}}<br>
            Dialogues in Dataset: ${{int(data.n_dialogues_in_dataset) || (
                Array.isArray(data.df_final) ? new Set(data.df_final).size : int(data.df_final)
            )}}<br>
            Utterances per Dialogue: ${{num(data.utterances_per_dialogue_mean)}} ± ${{num(data.utterances_per_dialogue_std)}}<br><br>

            <h3 style="margin-bottom:5px;">Clustering Metrics</h3>
            Number of States: ${{int(data.n_nodes)}}<br>
            Sentiment Cohesion: ${{num(data.fsc_mean)}}<br><br>

            <h3 style="margin-bottom:5px;">Flow Metrics</h3>
            Number of Transitions: ${{int(data.n_edges)}}<br>
            Initial Sentiment: ${{num(data.avg_SOD)}}<br>
            Final Sentiment: ${{num(data.avg_EOD)}}<br>
            Sentiment Variation: ${{num(data.sent_variation ?? (Number(data.avg_EOD) - Number(data.avg_SOD)))}}<br>
            Flow Density: ${{num(data.density)}}
            `;

        }})
        .catch(err => {{
            console.error(err);
            alert("Erro ao carregar o ficheiro.");
        }});
    }}
    document.getElementById("dotFileInput").addEventListener("change", function(event) {{
        const file = event.target.files[0];
        if (!file) return;
        document.getElementById("fileLabelText").textContent = file.name;
        enviarFicheiro();

        document.getElementById("file-subtitle").textContent = "Dialogue Flow Discovery for: " + file.name;

        const reader = new FileReader();
        reader.onload = function(e) {{
            const dotContent = e.target.result;
            renderNewDot(dotContent);
        }};
        reader.readAsText(file, 'iso-8859-1');
    }});
    function renderNewDot(dotString) {{
        container.innerHTML = "";  
        viz.renderSVGElement(dotString)
            .then(element => {{
                currentGraphElement = element;
                container.appendChild(element);
                
                element.querySelectorAll("g.node").forEach(node => {{
                    node.style.cursor = "pointer";
                const tooltip = document.getElementById("tooltip");
                let fixedTooltip = false;
            node.addEventListener("mouseover", (e) => {{
            if (fixedTooltip) return;
            const title = node.querySelector("title");
            if (!title) return;
                let falas = {clusters_speaker};         
                let falas_id = {cluster_id_speaker};  
                let new_person = title.textContent.split("->")[0].trim();
                let cluster = falas[new_person];
                let cluster_id = falas_id[new_person];
                let falas_speaker_js = (cluster && cluster[cluster_id]) ? cluster[cluster_id] : [];    
                const falas_speaker_js_curta = falas_speaker_js.filter(f => f.length <= 100);
                const falas_speaker_js_curta_aleatorias = falas_speaker_js_curta.sort(() => Math.random() - 0.5).slice(0, 5);
                if (new_person === "SOD"){{
                    tooltip.innerHTML  = "SOD";
                }}else if (new_person === "EOD"){{
                    tooltip.innerHTML  = "EOD";
                }}else {{
                tooltip.innerHTML = "<ul>" + 
                falas_speaker_js_curta_aleatorias.map(fala => "<li>" + fala + "</li>").join("") +
                "</ul>";

            }}
        const shape = node.querySelector("ellipse, polygon, circle, path");
        let fillColor = "#8B0000"; 
        if (shape) {{
            const stroke = shape.getAttribute("stroke");
            if (stroke && stroke !== "black") {{
                fillColor = stroke;
            }}
            const styleAttr = shape.getAttribute("style");
            if (styleAttr) {{
                const match = styleAttr.match(/stroke:\\s*(#[0-9a-fA-F]{{3,6}})/);
                if (match) {{
                    fillColor = match[1];
                }}
            }}
        }}

    tooltip.style.backgroundColor = fillColor;
    tooltip.style.display = "block";
    }});

    node.addEventListener("mousemove", (e) => {{
        if (!fixedTooltip) {{
            tooltip.style.left = (e.pageX + 15) + "px";
            tooltip.style.top = (e.pageY + 15) + "px";
        }}
    }});

    node.addEventListener("mouseout", () => {{
        if (!fixedTooltip) {{
            tooltip.style.display = "none";
        }}
    }});
                }});
                applyEdgeFilter(parseFloat(slider.value)/100);
                currentSpeaker = "__all__";
                applyFilters(parseFloat(slider.value) / 100, currentSpeaker);
            }})
            .catch(error => {{
                container.innerHTML = "<pre style='color:red'>" + error + "</pre>";
            }});
    }}

    document.querySelectorAll(".collapsible-header").forEach(header => {{
        header.addEventListener("click", () => {{
            const container = header.parentElement;
            container.classList.toggle("collapsed");
        }});
    }});
    document.querySelectorAll(".collapsible-header1").forEach(header => {{
        header.addEventListener("click", () => {{
            const container = header.parentElement;
            container.classList.toggle("collapsed");
        }});
    }});

    const dotCode = {dot_code_js};
    const container = document.getElementById("graph-container");
    const slider = document.getElementById("thresholdSlider");
    const sliderValue = document.getElementById("thresholdLabel");
    const thresholdMap = {threshold_json};
    const viz = new Viz();
    let currentGraphElement = null;
    let currentSpeaker = null;

    function applyEdgeFilter(threshold) {{
        if (!currentGraphElement) return;

        const visibleNodes = new Set();

        currentGraphElement.querySelectorAll("g.edge").forEach(edge => {{
            const label = edge.querySelector("text");
            const title = edge.querySelector("title");
            if (!label || !title) return;

            const valueStr = label.textContent;
            const value = parseFloat(valueStr.replace(',', '.'));

            if (!isNaN(value) && value >= threshold) {{
                edge.classList.remove("hidden");

                currentGraphElement.querySelectorAll("g.node > title").forEach(titleNode => {{
                    const nodeName = titleNode.textContent.trim();
                    if (title.textContent.includes(nodeName)) {{
                        visibleNodes.add(nodeName);
                    }}
                }});
            }} else {{
                edge.classList.add("hidden");
            }}
        }});

        currentGraphElement.querySelectorAll("g.node").forEach(node => {{
            const title = node.querySelector("title");
            if (!title) return;
            const nodeName = title.textContent.trim();
            if (visibleNodes.has(nodeName) || nodeName.toLowerCase() === "sod" || nodeName.toLowerCase() === "eod") {{
                node.classList.remove("hidden");
            }} else {{
                node.classList.add("hidden");
            }}

        }});
    }}
    
    viz.renderSVGElement(dotCode)
        .then(element => {{
            makeNodesDraggable(element); 
            applyEdgeFilter(parseFloat(slider.value) / 100);
            currentSpeaker = "__all__";
            applyFilters(parseFloat(slider.value) / 100, currentSpeaker);
            currentGraphElement = element;
            container.appendChild(element);
            element.querySelectorAll("g.node").forEach(node => {{
                node.style.cursor = "pointer";
                const tooltip = document.getElementById("tooltip");
                let fixedTooltip = false;
            node.addEventListener("mouseover", (e) => {{
            if (fixedTooltip) return;
            const title = node.querySelector("title");
            if (!title) return;
                let falas = {clusters_speaker};         
                let falas_id = {cluster_id_speaker};  
                let new_person = title.textContent.split("->")[0].trim();
                let cluster = falas[new_person];
                let cluster_id = falas_id[new_person];
                let falas_speaker_js = (cluster && cluster[cluster_id]) ? cluster[cluster_id] : [];    
                const falas_speaker_js_curta = falas_speaker_js.filter(f => f.length <= 100);
                const falas_speaker_js_curta_aleatorias = falas_speaker_js_curta.sort(() => Math.random() - 0.5).slice(0, 5);
                if (new_person === "SOD"){{
                    tooltip.innerHTML  = "SOD";
                }}else if (new_person === "EOD"){{
                    tooltip.innerHTML  = "EOD";
                }}else {{
                tooltip.innerHTML = "<ul>" + falas_speaker_js_curta_aleatorias.map(fala => "<li>" + fala + "</li>").join("") + "</ul>";
            }}
        const shape = node.querySelector("ellipse, polygon, circle, path");
        let fillColor = "#8B0000"; 
        if (shape) {{
            const stroke = shape.getAttribute("stroke");
            if (stroke && stroke !== "black") {{
                fillColor = stroke;
            }}
            const styleAttr = shape.getAttribute("style");
            if (styleAttr) {{
                const match = styleAttr.match(/stroke:\\s*(#[0-9a-fA-F]{{3,6}})/);
                if (match) {{
                    fillColor = match[1];
                }}
            }}
        }}

    tooltip.style.backgroundColor = fillColor;
    tooltip.style.display = "block";
    }});

    node.addEventListener("mousemove", (e) => {{
        if (!fixedTooltip) {{
            tooltip.style.left = (e.pageX + 15) + "px";
            tooltip.style.top = (e.pageY + 15) + "px";
        }}
    }});

    node.addEventListener("mouseout", () => {{
        if (!fixedTooltip) {{
            tooltip.style.display = "none";
        }}
    }});
        }});

                applyEdgeFilter(parseFloat(slider.value)/100);
                currentSpeaker = "__all__";
                applyFilters(parseFloat(slider.value) / 100, currentSpeaker);
            }})
            .catch(error => {{
                container.innerHTML = "<pre style='color:red'>" + error + "</pre>";
            }});

    let fixedTooltip = false;
    document.addEventListener("click", () => {{
        if (fixedTooltip) {{
            tooltip.style.display = "none";
            fixedTooltip = false;
            if (currentGraphElement) {{
                currentGraphElement.querySelectorAll("g.node").forEach(n => n.classList.remove("selected"));
            }}
        }}
    }});
    slider.addEventListener("input", () => {{
        const sliderIndex = parseInt(slider.value);
        const threshold = sliderIndex / 100;
        sliderValue.textContent = threshold.toFixed(2);
        applyEdgeFilter(threshold);
        if (thresholdMap[threshold.toFixed(2)]) {{
            Plotly.relayout('grafico', {{
                shapes: thresholdMap[threshold.toFixed(2)].shapes,
                annotations: thresholdMap[threshold.toFixed(2)].annotations
            }});
        }}
    }});

    function openSidebar_reset() {{
        document.getElementById("sidebar").style.width = "300px";
        document.getElementById("openSidebar").style.display = "none";
        document.getElementById("grafico").style.marginLeft = "300px";
        sidebar.style.transition = "all 0s ease";
    }}
    
    function openSidebar() {{
        document.getElementById("sidebar").style.width = "300px";
        document.getElementById("openSidebar").style.display = "none";
        document.getElementById("grafico").style.marginLeft = "300px";
    }}

    function closeSidebar() {{
        document.getElementById("sidebar").style.width = "0";
        document.getElementById("openSidebar").style.display = "block";
        document.getElementById("grafico").style.marginLeft = "0";
    }}
    
    function updateThreshold(sliderValueRaw) {{
        const threshold = parseFloat(sliderValueRaw) / 100;
        sliderValue.textContent = threshold.toFixed(2);
        applyFilters(threshold, currentSpeaker);
    }}

    function applySearchFilter(threshold,word){{
        const visibleNodes = new Set();
        const speakerLower = word.toLowerCase();

        
        currentGraphElement.querySelectorAll("g.edge").forEach(edge => {{
            const label = edge.querySelector("text");
            const title = edge.querySelector("title");
            const value = label ? parseFloat(label.textContent.replace(",", ".")) : NaN;
            const edgeTitle = title ? title.textContent : "";
            let source = null;
            let target = null;

            if (edgeTitle.slice(0, 3) === "SOD"){{
                source = "sod";
                target = edgeTitle.split("->")[2].split("(")[0].trim().toLowerCase();

            }}else{{
                const sourceMatch = edgeTitle.match(/^\s*([^->]+?)\s*->/);
                const targetMatch = edgeTitle.match(/\(\d+\)->\s*([^->]+)/g);
                if (sourceMatch && targetMatch?.length) {{
                    source = edgeTitle.split("->")[1].split("(")[0].trim().toLowerCase();
                    if (edgeTitle.split("->")[2].toLowerCase() === "eod"){{
                        target = "eod";
                    }}else{{
                        target =  edgeTitle.split("->")[3].split("(")[0].trim().toLowerCase();
                    }}
                }}
            }}
            const matchSpeaker = source === speakerLower || target === speakerLower;
            if (!isNaN(value) && value >= threshold) {{
                edge.classList.remove("hidden");
                if (!matchSpeaker) {{
                    edge.classList.add("faded");
                }} else {{
                    edge.classList.remove("faded");
                }}
                visibleNodes.add(source);
                visibleNodes.add(target);
            }} else {{
                edge.classList.add("hidden");
            }}
        }});

        currentGraphElement.querySelectorAll("g.node").forEach(node => {{
            const titleElement = node.querySelector("title");
            if (!titleElement) return;
            const nodeId = titleElement.textContent.trim().replace(/^"|"$/g, "");
            let nodeSpeaker = nodeId.split("->")[1]?.trim().toLowerCase();
            if(nodeSpeaker){{
                nodeSpeaker = nodeSpeaker.split(" ")[0];
            }}
            if (nodeId.toLowerCase() === "sod" || nodeId.toLowerCase() === "eod") {{
                node.classList.remove("faded");
            }} else if (nodeSpeaker === speakerLower) {{
                node.classList.remove("faded");
            }} else {{
                node.classList.add("faded");
            }}

            
        }});
    }}

    function resetOpacities() {{
        document.querySelectorAll(".node, .edge").forEach(el => {{
            el.classList.remove("faded");
            el.style.stroke = "";
            el.style.strokeWidth = "";
            el.style.opacity = "1";
            el.style.filter = "none";
        }});
    }}

    function applyFilters(threshold, speaker) {{
        if (!currentGraphElement) return;
        const visibleNodes = new Set();
        const speakerLower = speaker ? speaker.toLowerCase() : null;
        const allSpeakers = speaker === "__all__";
        currentGraphElement.querySelectorAll("g.edge").forEach(edge => {{
            const label = edge.querySelector("text");
            const title = edge.querySelector("title");
            const value = label ? parseFloat(label.textContent.replace(",", ".")) : NaN;
            const edgeTitle = title ? title.textContent : "";
            let source = null;
            let target = null;
            if (edgeTitle.slice(0, 3) === "SOD"){{
                source = "sod";
                target = edgeTitle.split("->")[1].trim().toLowerCase();
            }}else{{
                const sourceMatch = edgeTitle.match(/^\s*([^->]+?)\s*->/);
                const targetMatch = edgeTitle.match(/\(\d+\)->\s*([^->]+)/g);
                if (sourceMatch && targetMatch?.length) {{
                    source = sourceMatch[1].trim().toLowerCase();
                    const lastTargetMatch = targetMatch[targetMatch.length - 1];
                    target = lastTargetMatch.replace(/\(\d+\)->\s*/, "").trim().toLowerCase();
                }}
            }}
            const matchSpeaker = allSpeakers || source === speakerLower || target === speakerLower;
            if (!isNaN(value) && value >= threshold) {{
                edge.classList.remove("hidden");
                if (!matchSpeaker) {{
                    edge.classList.add("faded");
                }} else {{
                    edge.classList.remove("faded");
                }}
                visibleNodes.add(source);
                visibleNodes.add(target);
            }} else {{
                edge.classList.add("hidden");
            }}
        }});

        currentGraphElement.querySelectorAll("g.node").forEach(node => {{
            const titleElement = node.querySelector("title");
            if (!titleElement) return;
            const nodeId = titleElement.textContent.trim().replace(/^"|"$/g, "");
            const nodeSpeaker = nodeId.split("->")[0]?.trim().toLowerCase();
            const isVisible = visibleNodes.has(nodeId);
            if (nodeId.toLowerCase() === "sod" || nodeId.toLowerCase() === "eod") {{
                node.classList.remove("faded");
            }} else if (allSpeakers) {{
                node.classList.remove("faded");
            }} else {{
                if (nodeSpeaker === speakerLower) {{
                    node.classList.remove("faded");
                }} else {{
                    node.classList.add("faded");
                }}
            }}
        }});
    }}
    
    document.querySelectorAll('input[name="speaker"]').forEach(radio => {{
        radio.addEventListener("change", (e) => {{
            currentSpeaker = e.target.value;
            applyFilters(parseFloat(slider.value) / 100, currentSpeaker);
        }});
    }});

    document.getElementById("searchInput").addEventListener("input", (e) => {{
        const query = e.target.value.trim();
        if (query === "") {{
            resetOpacities(); 
        }} else {{
            applySearchFilter(parseFloat(slider.value) / 100,query);
        }}
    }});

    
    const file_name_inicial = "{file_name_inicial}";
    document.getElementById("file-subtitle").textContent = `Dialogue Flow Discovery for: {file_name_inicial}`;
    const btnReset = document.getElementById("btn-reset");
    const btnZoomIn = document.getElementById("btn-zoom-in");
    const btnZoomOut = document.getElementById("btn-zoom-out");
    const btnScreenshot = document.getElementById("btn-screenshot");
    const btnPan = document.getElementById("btn-pan");
    const btnZoomBox = document.getElementById("btn-zoom-box");
    const graphWrapper = document.getElementById("graph-wrapper");
    const graphContainer = document.getElementById("graph-container");
    let isPanning = false;
    let isDragging = false;
    let startX = 0, startY = 0;
    let translateX = 0, translateY = 0;
    let zoomLevel = 1;
    let isZoomBoxActive = false;
    let selectionRect = null;

    function desativaModos() {{
    isPanning = false;
    isZoomBoxActive = false;

    btnPan.classList.remove("active");
    btnZoomBox.classList.remove("active");

    btnPan.style.backgroundColor = "";
    btnZoomBox.style.backgroundColor = "";

    graphWrapper.style.cursor = "default";

    if (selectionRect) {{
        selectionRect.style.display = "none";
    }}
    }}

    function getRelativePositionInImage(e) {{
        const wrapperRect = graphWrapper.getBoundingClientRect();
        const mouseX = e.clientX - wrapperRect.left;
        const mouseY = e.clientY - wrapperRect.top;
        const x = (mouseX - translateX) / zoomLevel;
        const y = (mouseY - translateY) / zoomLevel;
        return {{ x, y }};
    }}

    function ativaPan() {{
    desativaModos();
    isPanning = true;
    btnPan.classList.add("active");
    btnPan.style.backgroundColor = "#0055aa";
    graphWrapper.style.cursor = "grab";

    if (selectionRect) {{
        selectionRect.style.display = "none";
    }}
    }}

    function ativaZoomBox() {{
    desativaModos();
    isZoomBoxActive = true;
    btnZoomBox.classList.add("active");
    btnZoomBox.style.backgroundColor = "#0055aa";
    graphWrapper.style.cursor = "crosshair";

    if (!selectionRect) {{
        selectionRect = document.createElement("div");
        selectionRect.id = "selection-rectangle";
        selectionRect.style.position = "absolute";
        selectionRect.style.border = "2px dashed #0055aa";
        selectionRect.style.backgroundColor = "rgba(0, 85, 170, 0.2)";
        selectionRect.style.pointerEvents = "none";
        graphWrapper.appendChild(selectionRect);
    }}
    selectionRect.style.display = "none";
    }}

    btnPan.addEventListener("click", () => {{
    if (isPanning) {{
        desativaModos();
    }} else {{
        ativaPan();
    }}
    }});

    btnZoomBox.addEventListener("click", () => {{
        if (isZoomBoxActive) {{
            desativaModos();
        }} else {{
            ativaZoomBox();
        }}
        }});

        graphWrapper.addEventListener("mousedown", (e) => {{
        if (!isZoomBoxActive) return;
        e.preventDefault();
        const pos = getRelativePositionInImage(e);
        startX = pos.x;
        startY = pos.y;
        selectionRect.style.left = `${{startX * zoomLevel + translateX}}px`;
        selectionRect.style.top = `${{startY * zoomLevel + translateY}}px`;
        selectionRect.style.width = "0px";
        selectionRect.style.height = "0px";
        selectionRect.style.display = "block";
    }});

    graphWrapper.addEventListener("mousemove", (e) => {{
        if (!isZoomBoxActive || selectionRect.style.display === "none") return;
        const pos = getRelativePositionInImage(e);
        const rectX = Math.min(pos.x, startX);
        const rectY = Math.min(pos.y, startY);
        const rectWidth = Math.abs(pos.x - startX);
        const rectHeight = Math.abs(pos.y - startY);
        selectionRect.style.left = `${{rectX * zoomLevel + translateX}}px`;
        selectionRect.style.top = `${{rectY * zoomLevel + translateY}}px`;
        selectionRect.style.width = `${{rectWidth * zoomLevel}}px`;
        selectionRect.style.height = `${{rectHeight * zoomLevel}}px`;
    }});

    graphWrapper.addEventListener("mouseup", (e) => {{
        if (!isZoomBoxActive || selectionRect.style.display === "none") return;
        selectionRect.style.display = "none";
        const pos = getRelativePositionInImage(e);
        const x1 = Math.min(startX, pos.x);
        const y1 = Math.min(startY, pos.y);
        const x2 = Math.max(startX, pos.x);
        const y2 = Math.max(startY, pos.y);
        const selectedWidth = x2 - x1;
        const selectedHeight = y2 - y1;
        if (selectedWidth < 10 || selectedHeight < 10) {{
            return; 
        }}
        zoomToArea(x1, y1, selectedWidth, selectedHeight);
        desativaModos();
    }});


    function zoomToArea(x, y, width, height) {{
    const wrapperRect = graphWrapper.getBoundingClientRect();
    const scaleX = wrapperRect.width / width;
    const scaleY = wrapperRect.height / height;
    const newScale = Math.min(scaleX, scaleY);
    translateX = -x * newScale;
    translateY = -y * newScale;
    zoomLevel = newScale;
    graphContainer.style.transform = `translate(${{translateX}}px, ${{translateY}}px) scale(${{zoomLevel}})`;
    graphContainer.style.transformOrigin = "0 0";
    }}

    graphWrapper.addEventListener("mousedown", (e) => {{
    if (!isPanning) return;
    e.preventDefault();
    isDragging = true;
    startX = e.clientX - translateX;
    startY = e.clientY - translateY;
    graphWrapper.style.cursor = "grabbing";
    }});
    document.addEventListener("mousemove", (e) => {{
    if (!isPanning || !isDragging) return;
    translateX = e.clientX - startX;
    translateY = e.clientY - startY;
    graphContainer.style.transform = `translate(${{translateX}}px, ${{translateY}}px) scale(${{zoomLevel}})`;
    graphContainer.style.transformOrigin = "0 0";
    }});

    document.addEventListener("mouseup", () => {{
    if (!isPanning) return;

    isDragging = false;
    graphWrapper.style.cursor = "grab";
    }});

    window.addEventListener("DOMContentLoaded", () => {{
        const shouldReopenMenu = localStorage.getItem("menuOpen") === "true";
        if (shouldReopenMenu) {{
            openSidebar_reset(); 
        }}
        localStorage.removeItem("menuOpen"); 
    }});

    btnReset.addEventListener("click", () => {{
        const openSidebarBtn = document.getElementById("openSidebar");
        const isMenuOpen = openSidebarBtn.style.display === "none";
        localStorage.setItem("menuOpen", isMenuOpen ? "true" : "false");
        location.reload();
        }});

    btnZoomIn.addEventListener("click", () => {{
        zoomLevel = Math.min(zoomLevel + 0.1, 3);
        graphWrapper.style.transform = `scale(${{zoomLevel}})`;
        graphWrapper.style.transformOrigin = "50% 50%";
    }});

    btnZoomOut.addEventListener("click", () => {{
        zoomLevel = Math.max(zoomLevel - 0.1, 0.5);
        graphWrapper.style.transform = `scale(${{zoomLevel}})`;
        graphWrapper.style.transformOrigin = "50% 50%";
    }});

    btnScreenshot.addEventListener("click", () => {{
        const wrapper = document.getElementById("graph-wrapper");
        const rect = wrapper.getBoundingClientRect(); 
        html2canvas(document.body, {{
            backgroundColor: "#ffffff",
            scale: 2,
            useCORS: true,
            windowWidth: window.innerWidth,
            windowHeight: window.innerHeight
        }}).then(canvas => {{
            const scale = 2; 
            const cropX = rect.left * scale;
            const cropY = rect.top * scale;
            const cropWidth = rect.width * scale;
            const cropHeight = rect.height * scale;
            const croppedCanvas = document.createElement("canvas");
            croppedCanvas.width = cropWidth;
            croppedCanvas.height = cropHeight;
            const ctx = croppedCanvas.getContext("2d");
            ctx.drawImage(
                canvas,
                cropX, cropY,
                cropWidth, cropHeight,
                0, 0,
                cropWidth, cropHeight
            );
            const link = document.createElement("a");
            link.download = "fluxo_visual_visivel.png";
            link.href = croppedCanvas.toDataURL("image/png");
            link.click();
        }}).catch(error => {{
            alert("Erro ao capturar imagem visível.");
            console.error(error);
        }});
    }});
    
    function initTooltips(svgElement) {{
    svgElement.querySelectorAll("g.node").forEach(node => {{
        node.style.cursor = "pointer";
        const tooltip = document.getElementById("tooltip");
        let fixedTooltip = false;

        node.addEventListener("mouseover", (e) => {{
            if (fixedTooltip) return;
            const title = node.querySelector("title");
            if (!title) return;

            let falas = clusters_speaker;         
            let falas_id = cluster_id_speaker;  
            let new_person = title.textContent.split("->")[0].trim();
            let cluster = falas[new_person];
            let cluster_id = falas_id[new_person];
            let falas_speaker_js = (cluster && cluster[cluster_id]) ? cluster[cluster_id] : [];    

            const falas_speaker_js_curta = falas_speaker_js.filter(f => f.length <= 100);
            const falas_speaker_js_curta_aleatorias = falas_speaker_js_curta.sort(() => Math.random() - 0.5).slice(0, 5);

            if (new_person === "SOD"){{
                tooltip.innerHTML = "SOD";
            }} else if (new_person === "EOD"){{
                tooltip.innerHTML = "EOD";
            }} else {{
                tooltip.innerHTML = "<ul>" + falas_speaker_js_curta_aleatorias.map(fala => "<li>" + fala + "</li>").join("") +"</ul>";
            }}

            const shape = node.querySelector("ellipse, polygon, circle, path");
            let fillColor = "#8B0000"; 
            if (shape) {{
                const stroke = shape.getAttribute("stroke");
                if (stroke && stroke !== "black") {{
                    fillColor = stroke;
                }}
                const styleAttr = shape.getAttribute("style");
                if (styleAttr) {{
                    const match = styleAttr.match(/stroke:\s*(#[0-9a-fA-F]{{3,6}})/);
                    if (match) {{
                        fillColor = match[1];
                    }}
                }}
            }}

            tooltip.style.backgroundColor = fillColor;
            tooltip.style.display = "block";
        }});

        node.addEventListener("mousemove", (e) => {{
            if (!fixedTooltip) {{
                tooltip.style.left = (e.pageX + 15) + "px";
                tooltip.style.top = (e.pageY + 15) + "px";
            }}
        }});

        node.addEventListener("mouseout", () => {{
            if (!fixedTooltip) {{
                tooltip.style.display = "none";
            }}
        }});
    }});
}}

</script>
</body>
</html>
"""

caminho_html = os.path.join('./Resultados/'  'grafo_interativo_2024.html')
os.makedirs(os.path.dirname(caminho_html), exist_ok=True)

with open(caminho_html, 'w', encoding='utf-8') as f:
    f.write(html_content)

webbrowser.open('file://' + os.path.realpath(caminho_html))

True